In [57]:
import ssl
import os
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
os.environ["REQUESTS_CA_BUNDLE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")
os.environ["SSL_CERT_FILE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")

## Model Download

In [38]:
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org cmake
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 24.0 from C:\ProgramData\Anaconda3\envs\mchatbot\lib\site-packages\pip (python 3.8)

  Running command pip subprocess to install build dependencies
    Using cached setuptools-72.1.0-py3-none-any.whl.metadata (6.6 kB)
    Using cached scikit_build-0.18.0-py3-none-any.whl.metadata (17 kB)
    Using cached cmake-3.30.1-py3-none-win_amd64.whl.metadata (6.1 kB)
    Using cached ninja-1.11.1.1-py2.py3-none-win_amd64.whl.metadata (5.4 kB)
    Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
    Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
    Using cached tomli-2.0.1-py3-none-any.whl.metadata (8.9 kB)
    Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached setuptools-72.1.0-py3-none-any.whl (2.3 MB)
  Using cached scikit_build-0.18.0-py3-none-any.whl (85 kB)
  Using cached cmake-3.30.1-py3-none-win_amd64.whl (35.7 MB)
  Using cached ninja-1.11.1.1-py2.py3-none-win_amd64.whl (312 kB)
  Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached packaging-24.1-py3-n


     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ------ --------------------------------- 0.3/1.7 MB 8.9 MB/s eta 0:00:01
     ---------------- ----------------------- 0.7/1.7 MB 9.1 MB/s eta 0:00:01
     -------------------------------- ------- 1.4/1.7 MB 11.0 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Link requires a different Python (3.8.19 not in: '>=3.9'): https://files.pythonhosted.org/packages/26/de/437a60a69f7fd0c79264530a97787b2ac7394616e3661236201518f8a47d/numpy-1.25.0rc1.tar.gz (from https://pypi.org/simple/numpy/) (requires-python:>=3.9)
  Link requires a different Pyth

In [58]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [59]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [60]:
# code for download the model from huggingface
# model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename, local_dir=r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Learnings\GitHub\End-to-End-Medical-Chatbot-using-LLM\model")

In [61]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [62]:
PINECONE_API_KEY = "7f6b7506-8283-42ff-8fce-c62007e0dfa1"
PINE_CONE_HOST = "https://medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io"

In [63]:
## Extract the data from pdf
def load_pdf(data):
    loader =DirectoryLoader(data, glob="*.pdf", loader_cls=PyMuPDFLoader)

    documents = loader.load()
    return documents

In [64]:
extracted_data = load_pdf('data/')

In [65]:
# extracted_data

In [66]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [67]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

5779


In [68]:
## download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [69]:
embeddings = download_hugging_face_embedding()

In [70]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [71]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


## PineCone vectorDB initialisation

In [72]:
PINECONE_API_KEY = "7f6b7506-8283-42ff-8fce-c62007e0dfa1"
PINE_CONE_HOST = "https://medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io"

In [73]:
from pinecone.grpc import PineconeGRPC as Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [74]:
index_name = "medical-chatbot"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
index = pc.Index("medical-chatbot")



In [77]:
index.describe_index_stats()
# vector_lst = []
# for i in text_chunks:
#     vector_lst.append(embeddings.embed_query(i.page_content))

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

In [103]:
# Iterate over words and their embeddings
import tqdm
for i, t in enumerate(text_chunks):
    # Upsert each word embedding into the index
    index.upsert(vectors=[{"id": str(i), "values": embeddings.embed_query(t.page_content), "metadata": {"text": t.page_content}}])

In [104]:
index.describe_index_stats()


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5779}},
 'total_vector_count': 5779}

In [115]:
def query(text):
    query_embedding = embeddings.embed_query(text)
    print(query_embedding)
    query_result = index.query(query_embedding, top_k=2, top_n=1)
    return query_result

In [132]:
query_result = index.query(
    vector=embeddings.embed_query("what are allergies?"),
    top_k=2,
    include_values=True,
    include_metadata=True,
)

In [133]:
print(query_result["matches"][0]['metadata']['text'])

reaction. Allergic rhinitis is characterized by an itchy,
runny nose, often with a scratchy or irritated throat due
to post-nasal drip. Inflammation of the thin membrane
covering the eye (allergic conjunctivitis) causes redness,
irritation, and increased tearing in the eyes. Asthma caus-
es wheezing, coughing, and shortness of breath. Symp-
toms of food allergies depend on the tissues most sensi-
tive to the allergen and whether the allergen spread sys-
